### Global Housing Analysis: Factors Influencing Housing Trends
#### [Link to Dataset](https://www.kaggle.com/datasets/emanfatima2025/international-house-sales-and-buyer-loan-records)

* Objective: Observe trends across the global housing market to see how factors, such as salary and monthly expenses, influence the loan amount and tenure rates in different countries along with what the house price signifies about the country’s housing economy as a whole

* Introduction: This dataset gathered information from 200,000 records of home purchases made globally along with loan approval decisions. Such information included location, house features, area, and loan information.
    * Because of the bulk of this dataset, according to the creator, it can be used by researchers and data scientists to investigate patterns of urban growth, risk assessment, financial decision-making, and housing trends.

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
#Importing csv file
df = pd.read_csv('Global_House.csv')
df

In [ ]:
# First 5 Rows
df.head()

In [ ]:
# Last 5 Rows
df.tail()

In [ ]:
# Checking the datatype for each column 
df.info()

In [ ]:
# Checking for null values
df.isnull().sum()

In [ ]:
# Checking for duplicates
df.duplicated().sum()

In [ ]:
# Creating a new variable for the copied dataframe
house = df.copy()
house

#### Data Cleaning
* I deleted the 'property_id', 'city', 'rooms', 'satisfaction_score', 'neighbourhood_rating', 'connectivity_score' columns then the 'bathrooms', 'garage', 'garden', 'crime_cases_reported', 'legal_cases_on_property', and 'previous_owners' columns since I am not looking at specific house information but rather the bigger picture for housing trends

In [ ]:
# Dropping unnecessary columns for this analysis pt.1
house.drop(['property_id', 'city', 'rooms', 'satisfaction_score', 'neighbourhood_rating', 'connectivity_score'], axis = 1, inplace = True)
house

In [ ]:
#Dropping unnecessary columns pt.2 
house.drop(['bathrooms', 'garage', 'garden', 'crime_cases_reported', 'legal_cases_on_property', 'previous_owners'], axis = 1, inplace=True)
house

* Next, I sorted the remaining values of "house" by price 

In [ ]:
# Sorting houses by price
house.sort_values('price')

* Then, with 13 columns left, I decided to filter the ones I needed instead of deleting the rest as I may need them in my analysis; that said, I created a new variable, “house_filtered,” and put the 'country', 'price', 'property_type', 'customer_salary', 'loan_amount', 'down_payment', 'loan_tenure_years', and 'monthly_expenses' columns in it.

In [ ]:
# Filtering the data for the columns specifing needed for this analysis
house_filtered = house[['country', 'price', 'property_type', 'customer_salary', 'loan_amount', 'down_payment', 'loan_tenure_years', 'monthly_expenses']]
house_filtered

* I then sorted these values also by price to see the lowest price and highest price of a house in the dataset while also showing the associated salary and monthly expenses information. 

In [ ]:
house_filtered = house_filtered.sort_values('price')
house_filtered

* With the eight columns I felt I needed, I decided to look at the statistical information using .describe()

In [ ]:
# Getting statistical information
house_filtered.describe()

* The house prices column had a close dispersion with the 1st, 2nd, and 3rd quartile having about a 500,000-700,000 difference between them. In this column, the minimum and maximum were the outliers by from the US and Canada being 56,288 and 4,202,732, and the mean  being 1,215,365. However, what shocked me the most was the mean for the loan amount and down payment columns being 759,758 and 455,606 respectively, making me question why such averages were so high and if they correlated with the customer's salary and the house price.

* To test, I used information from the US and Canada to analyze what effect a customer's salary plays 

In [ ]:
# Houses in the US
house_us= house_filtered[house_filtered['country']== 'USA']
house_us = house_us.sort_values('customer_salary')
house_us

* In the US, to pay for a 2.9 million dollar studio on a 10,015 dollar salary with approximately 5,000 dollars in monthly expenses, one customer had a down payment of 1.6 million and a loan amount of 1.3 million on a 20 year tenure.

In [ ]:
# Houses in Canada
house_ca= house_filtered[house_filtered['country']== 'Canada']
house_ca = house_ca.sort_values('customer_salary')
house_ca

* In Canada, to buy a 1.3 million independent house at the same salary but 10,170 in monthly expenses, one customer had a down payment of 698,493 and had a loan of 674,148 at the same loan tenure.

* Though I believe there are outside factors within each country influencing the price of the properties, I had a hypothesis that there is still a correlation between one’s salary, the loan amount, and down payment offers, leading me to ponder that when considering visualizations. 

### Visualizations

#### Heatmap
* Using the hypothesis above, I decided to create a heatmap to see all correlations with the numerical information in the dataset. 

In [ ]:
#Heatmap
corr_matrix= house_filtered.corr(numeric_only=True)

In [ ]:
# Creating Heatmap
ax = sns.heatmap(
    corr_matrix, annot=True, fmt=".2f",
    cmap="twilight", center=0,
    linewidths=0.5, cbar_kws={"label": "Correlation"}
)
plt.title("Correlation Heat Map")
plt.tight_layout()
plt.savefig('HouseHeatmap.jpg')
plt.show()

 * It shows that there is a weak correlation with the customers’ salary, the loan amount (.23), and down payment (.20), disapproving my earlier hypothesis. However, there are interesting relationships within this map as the house price has a strong positive relationship with the loan amount (.94) and the down payment (.85). There is also another strong positive relationship with the down payment and the loan amount (.62).

 * Revealing these relationships help indicate trends on what factors influence house purchasing globally
-----
#### Bar Chart
* To get an overall picture, I decided to do a bar plot showcasing the average house price of each country. To do this, I first created a variable, “dollar,” that incorporated a groupby function that gathers all the countries in the dataset and aggregates the mean of each by the price column.
  
* From there, I created a user defined variable that will create a new label for the mean so the graph will look visually appealing. I then added a new column, “Label,” to dollar.

In [ ]:
#Adding a label to the price to make the graph look visually appealing
def format_number(n):
    if n >= 1_000_000:
        return f"{n/1_000_000:.1f}M"
    elif n >= 1_000:
        return f"{n/1_000:.0f}K"
    else:
        return str(n)

In [ ]:
# Grouping data specifically for country and house price
dollar = house_filtered.groupby('country',as_index=False)['price'].mean().round().sort_values(by = 'price',ascending=False, inplace=False)
dollar

In [ ]:
#Applying the defined variable to dollar
dollar["Label"] = dollar["price"].apply(format_number)
dollar

In [ ]:
#px barplot showing the average house prices globally
monies = px.bar(dollar, 
                x = 'country', 
                y = 'price', 
                text = 'Label',
                title = 'Average House Prices Globally',
                orientation ='v')
monies.update_layout(
    yaxis = dict(title = 'House Prices'),
    xaxis = dict(title = 'Country'))
monies.write_html("House_Bar.html")
monies.show()

* Using this graph, the top five countries are Singapore, United Arab Emirates (UAE), the United States, Japan, and France as they all have an average house price higher than 1.3 million (though the UK says 1.3 million, that is simply an approximation as the real number is around 1.29 million)

* Since house prices reflect the current state of the economy, there are questions behind if this information truly reflects that state of each country

----

#### Line Plot

* I decided to compare the country with the highest house price and the lowest house price using a line plot. Using Singapore and India, I compared salary and monthly expenses using the variable "df_houses" 

In [ ]:
# Compiling information from India and Singapore
df_houses = house_filtered[house_filtered['country'].isin(['India', 'Singapore'])]
df_houses = df_houses.sort_values('price')
df_houses

In [ ]:
#Creating line plot
fig = px.line(
    df_houses,
    x='customer_salary',
    y='monthly_expenses',
    color='country',
    title = 'Monthy Expenses in Relation to Customer Salary in India vs Singapore',
    )
fig.update_layout(
    yaxis = dict(title = 'Monthly Expenses'),
    xaxis = dict(title = 'Salary'))
fig.write_html("India_Singapore_line.html")
fig.show()

* In this graph, the salary for India has a smaller range, but with that, there is also a smaller range for one’s monthly expenses. Conversely, Singapore has a wider salary range and a wider monthly expense gap. This leads to a trend that relativity plays a role into the expenses one would have to pay.

* To clarify, the correlation is not one-to-one but rather non-exclusive, leading to how there are some people within this graph having a 20,000 dollar salary, but also owing 20,000 dollars in monthly expenses.

-----
 #### Scatterplot

 * Following the insights found using the line plot, I decided to do a scatterplot on the same two country comparing the salary with the house price

In [ ]:
# Creating scatterplot 
versus = px.scatter(df_houses, x="customer_salary", y="price", color="country",
                   title = 'Highest Average vs Lowest Average House Price in Relation to Salary')
versus.update_layout(
    yaxis = dict(title = 'House Prices'),
    xaxis = dict(title = 'Salary'))
versus.write_html("India_Singapore_scatter.html")
versus.show()

* Using this graph, this trend of relativity showed out again as it shows that there are customers who had less than a 20,000 dollar salary paying for a house over 4 million dollars.

* To confirm this theory, I decided to use a different country: the US. By using the "house_us" variable used earlier, I created another scatterplot to see if this trend continued. When hovering over the dots, I decided to put the down payment of the house required along with color coding based on the loan tenure for each house.

In [ ]:
# Creating U.S. scatterplot
coin = px.scatter(house_us,
               x='price',
               y='customer_salary',
               color ='loan_tenure_years',
               hover_name ='down_payment',
               title = 'Customer Salary Compared to House Price in the US',
               width = 750,
               height =1000)
coin.update_layout(
    yaxis = dict(title = 'Salary'),
    xaxis = dict(title = 'House Price'),
coloraxis_colorbar=dict(title= "Loan Tenure"))
coin.write_html("US_House_Scatter.html")

coin.show()

* As shown here, the trend still stands as there are some people with less than a 20,000 salary paying for a 3 million dollar house.

* Because of the abnormality of such scatterplots being in this congealed state, it ultimately led me to believe that this is a trend.

* However, to look closer into why this is the case, I unfortunately needed more information than what was within the dataset.

    * Aspects such as affordability/affordability rate along with how much the customer actually pays for the house (i.e. at closing and for the mortgage each month) would have contributed to analyzing why the scatterplots look like this and the overall implications concerning the global housing market. 
----
#### Histogram

* In a final push to find another trend within the dataset, I created a histogram measuring the counts of different loan tenures based on monthly expenses within the US

In [ ]:
# Creating histogram
dinero = px.histogram(data_frame = house_us, x = 'monthly_expenses',
                     nbins=25,
                     title = 'Monthly Expenses & Associated Loan Tenures in the US',
                     hover_name = 'price',
                     histfunc = 'count',
                     color='loan_tenure_years')
dinero.update_layout(
    yaxis = dict(title = 'Count'),
    xaxis = dict(title = 'Monthly Expenses'))
dinero.write_html("US_Month_Expenses_Hist.html")
dinero.show()

* When looking into those whose monthly expenses are 2,000 dollars, it seems that customers choose the 30 year tenures more than the other options in that category

* This insight leads to the inference that the lower ones expenses are, the chance for a longer loan tenure.
    * Unfortunately, I am unable to further delve into the reasons customers opt for a longer loan tenure as no column information corresponds with digging deeper into this insight.
        * If the dataset included interest rates, this histogram would have allowed me to use it to count the different loan tenure options within the various loan interest rates to see if interest rates affect loan tenure and, ultimately, the customer's decision. 
----
#### Geo Map

* Finally, I created a geo map, meant to show the different house prices and specifics in each country using the dollar variable from earlier after applying the variable “country_to_iso” that programed Python to locate each country on the map using its 3 letter code 

In [ ]:
# Helps Python to locate countries on the geo map based on 3-letter code
country_to_iso = {
    'Australia': 'AUS',
    'Brazil': 'BRA',
    'Canada': 'CAN',
    'USA': 'USA',
    'UAE': 'ARE',
    'China': 'CHN',
    'Japan': 'JPN',
    'India': 'IND',
    'South Africa': 'ZAF',
    'UK': 'GBR',
    'Germany': 'DEU',
    'Singapore':'SGP',
    'France': 'FRA'}


In [ ]:
# Applying country_to_iso to the dollar variable
dollar['iso_code'] = dollar['country'].map(country_to_iso)
dollar

In [ ]:
# Creating geo map
globalz = px.choropleth(dollar,
                    locations='iso_code',
                    color='Label',
                    hover_name='country',
                    projection='natural earth',
                    color_continuous_scale='thermal',
                    title = 'Average House Price by Country')
globalz.write_html('HouseGlobe.html')
globalz.show()

* Due to the lack of interest or inflation rates in each country, the graph was only able to show the average house prices as emphasized earlier by the bar chart.

* If the dataset had more information on the country’s economy itself, especially concerning the housing market, the geo map’s capabilities would have been enhanced and allowed for deeper research into the trends and inferences referenced above.

* Ultimately, though this research ended up being a snapshot for global housing trends, there were still key insights that could lead into further research in displaying the entire picture of the global housing market.

----

#### Final csv

* To save the dataframe, "house_filtered", that I used in this analysis, I decided to make it as a new csv called "HouseTrend."

In [ ]:
house_filtered.to_csv('HouseTrend.csv', index = False)